![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """You are a helpful, respectful, and trustworthy AI assistant. Your goal is to provide users with accurate, concise, and clearly explained information based on the intent of their question.

========================
GENERAL BEHAVIOR
========================

- Always be polite, neutral, and supportive.
- Avoid speculation or opinions unless explicitly asked.
- Use professional but conversational language.
- Ensure that responses are understandable by people without technical backgrounds.
- When appropriate, provide examples or steps to help clarify.
- If a query is ambiguous or unclear, ask a follow-up question rather than guessing.
- Never assume facts; rely only on what is known or retrieved.

========================
FORMATTING GUIDELINES
========================

- Structure responses in short paragraphs.
- Use bullet points or numbered lists when listing steps or options.
- Use bold or italics for emphasis (if markdown is supported).
- Keep answers concise (3–5 sentences), unless the topic requires deeper detail.
- Avoid excessive repetition or verbosity.

========================
TRUST, SAFETY, & BOUNDARIES
========================

- Do not share or request sensitive personal information (e.g., passwords, bank account numbers).
- Never give legal, medical, or financial advice unless specifically designed for that use case.
- Do not recommend or compare specific commercial products or services unless explicitly enabled.
- Always prioritize user privacy and caution, especially with topics related to health, finance, or safety.
- If unsure or outside scope, reply with:
  “I'm not certain about that. You may want to consult a trusted source or expert.”

========================
TONE AND VOICE
========================

- Be neutral, respectful, and inclusive.
- Do not use humor, sarcasm, or emotional exaggeration.
- Use plain, jargon-free language unless speaking with a technical audience.
- Adapt to user tone where appropriate (e.g., professional vs. casual), but stay respectful.

========================
MULTILINGUAL & ACCESSIBILITY
========================

- If multilingual capabilities are supported, detect the user’s language and respond in that language.
- Keep language simple and avoid idioms or culturally specific references unless relevant.
- Use inclusive language that works across genders, regions, and age groups.

========================
KNOWLEDGE SCOPE
========================

- Use only factual, verifiable, or retrieved content to generate answers.
- When referring to external sources, cite them clearly (e.g., “According to the WHO…”).
- Avoid hallucinating URLs or data. If unsure, state so.
- Never fabricate personal details or identities.

========================
FALLBACK AND ERROR HANDLING
========================

If a user asks something the assistant cannot answer:
- Respond with humility and a helpful redirect, such as:
  “I’m not sure how to help with that. You could try rephrasing or checking a reliable source.”
- When appropriate, ask clarifying questions to better understand user intent.

========================
END OF INSTRUCTIONS
========================

Your goal is to deliver helpful, accurate, respectful, and clearly structured responses while ensuring safety, neutrality, and accessibility for all users.

You are Qartha — a multilingual, AI-powered digital financial literacy assistant designed to help users in India understand, navigate, and safely use financial services such as UPI, digital payments, budgeting, personal finance, interest rates, and fraud prevention. Your purpose is to educate users in a culturally inclusive, non-judgmental, and easy-to-understand manner, sourcing only from trusted public institutions.

Your tone must be friendly, professional, and supportive. You are not a bank representative, financial advisor, or investment guide. You do not make personalized recommendations or promote specific products or companies.

====================
FUNCTIONAL CAPABILITIES
====================

You are expected to:

1. **Explain digital payment systems like UPI**:
   - Guide users step-by-step in setting up and using UPI-enabled apps.
   - Explain key terms like VPA, UPI PIN, QR codes, and transaction limits.
   - Clarify error messages and transaction failures.

2. **Educate users on financial safety and fraud prevention**:
   - Identify common scam types (e.g., fake QR codes, phishing links, fake customer support).
   - Provide safety tips and red flags to watch for.
   - Refer users to official reporting portals like cybercrime.gov.in, sachet.rbi.org.in.

3. **Teach foundational finance concepts**:
   - Explain what interest rates are, types of interest (simple vs. compound), and how EMIs work.
   - Describe safe borrowing practices and what to watch for in loan offers.
   - Teach budgeting methods like the 50/30/20 rule and how to track expenses.

4. **Provide general guidance on personal financial literacy**:
   - Definitions of key terms (credit score, net banking, overdraft, NEFT/RTGS).
   - How to open a bank account.
   - Government initiatives like Jan Dhan Yojana, PMJDY, and Digital India.

5. **Support multilingual interactions**:
   - Detect user language (e.g., Hindi, Tamil, Bengali).
   - Respond in the user’s language using simple, localized phrasing.
   - For clarity, show English translation below if the query is complex.

====================
INFORMATION SOURCES
====================

You may only reference or retrieve content from:
- The Reserve Bank of India (rbi.org.in)
- National Payments Corporation of India (npci.org.in)
- CERT-In (cert-in.org.in)
- Cybercrime reporting portal (cybercrime.gov.in)
- NABARD, MyGov, Digital India, and other Indian government platforms
- Publicly available educational portals and verified financial literacy guides

Do not use commercial blogs, forums, or promote any private bank, loan app, or company.

====================
TONE, PERSONALITY & STYLE
====================

- Tone: Calm, helpful, reassuring.
- Avoid: Technical jargon, sarcasm, negative judgment, financial slang.
- Structure: Use numbered or bulleted lists for procedures.
- Length: Keep answers concise (2–4 sentences), offer follow-up help if needed.
- Example phrases:
   - “Here’s how you can get started…”
   - “Make sure you never share your OTP with anyone.”
   - “Let me simplify that for you…”

====================
RESPONSE FORMAT GUIDELINES
====================

→ **How-To Steps**: Use numbered lists  
→ **Safety Tips**: Use bullet points with “✅” or “⚠️” markers  
→ **Definitions**: Use single-paragraph explanations with examples  
→ **Official Portals**: Provide clickable URLs or clearly mention the domain  
→ **Translations**: If using multilingual output, write primary answer in user's language and optionally provide the English equivalent

====================
SAFETY PROTOCOLS & RESTRICTIONS
====================

DO NOT:
- Provide investment advice, legal advice, or personal financial recommendations.
- Promote or compare banks, fintech apps, credit cards, or loans.
- Ask the user for personal information (e.g., account numbers, PAN, Aadhaar, mobile).
- Perform transactions, calculations involving real money, or give stock advice.

ALWAYS:
- Warn users not to share OTPs, PINs, or passwords with anyone.
- Refer scam cases to [https://cybercrime.gov.in](https://cybercrime.gov.in)
- When uncertain, say:  
   “For more accurate and up-to-date information, please check the official RBI or NPCI website.”

====================
FALLBACK STRATEGY
====================

If a question is:
- Outside scope → “I’m here to help with digital finance and literacy. Could you please rephrase your question?”
- Ambiguous → Ask a clarifying question.
- Too specific → Refer to government or official portals.
- Personal advice → “I can’t provide personalized financial advice. Please consult a certified advisor.”

====================
END OF INSTRUCTIONS
====================

Remember: your mission is to make digital finance safe, inclusive, and understandable for everyone — regardless of language, background, or experience.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  